In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [4]:
import torch
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
# imports from captum library
from captum.attr import IntegratedGradients, GradientShap, DeepLift, NoiseTunnel
import shap


In [5]:
from utils import helper
from ml.preprocess import data
from ml.models.base.brisk_model_lightning import BriskModel

ModuleNotFoundError: No module named 'ml.models.base.brisk_model_lightning'

In [11]:

X, y = data.get_dataset(f_name = '20220319_covid_merge_processed.csv' , sep = ',')

X_tilda = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X_tilda, y, test_size=0.25, random_state=123)

X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.25, random_state=123)


In [6]:
# nn_model = keras.models.load_model(brisk_neural_net.MODEL_SAVE_PATH)

import pickle
MODEL_SAVE_PATH = "/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/models/saved/base/brisk_nn_pytorch/"


with open(MODEL_SAVE_PATH+"best_pytorch.pickle", "rb") as fin:
    briskmodel_loaded = pickle.load(fin)



In [12]:
briskmodel_loaded.eval()

Sequential(
  (0): Linear(in_features=40, out_features=222, bias=True)
  (1): ReLU()
  (2): Linear(in_features=222, out_features=180, bias=True)
  (3): ReLU()
  (4): Linear(in_features=180, out_features=121, bias=True)
  (5): ReLU()
  (6): Linear(in_features=121, out_features=160, bias=True)
  (7): ReLU()
  (8): Linear(in_features=160, out_features=216, bias=True)
  (9): ReLU()
  (10): Linear(in_features=216, out_features=239, bias=True)
  (11): ReLU()
  (12): Linear(in_features=239, out_features=1, bias=True)
  (13): ReLU()
)

### IntegratedGradients

In [13]:
df_tensor = helper.df_to_tensor(X_train)
# df_tensor = df_tensor.movedim(0,-1)

In [14]:
df_tensor.shape

torch.Size([143, 40])

In [15]:
ig = IntegratedGradients(briskmodel_loaded)
ig_attr = ig.attribute(df_tensor, n_steps = 500)


In [59]:
ig_attr_df_shapley = pd.DataFrame(ig_attr.numpy(), columns=X_train.columns)
ig_attr_df_shapley = ig_attr_df_shapley.abs().mean().sort_values(ascending=False)

In [45]:
ig_attr_df_shapley_list = ig_attr_df_shapley.abs().mean().sort_values(ascending=False).values
ig_attr_df_shapley_list

array([1.02273343e+09, 8.47671604e+08, 2.23971596e+07, 1.03326653e+07,
       7.18673734e+04, 2.19613393e+04, 1.16263309e+03, 5.98648609e+02,
       3.59338653e+01, 2.80964134e+01, 8.95898574e+00, 4.31250295e+00,
       3.98550994e+00, 2.84612252e+00, 2.78459601e+00, 2.71920249e+00,
       1.69220587e+00, 1.08083201e+00, 8.60945500e-01, 7.53281921e-01,
       6.83303135e-01, 4.28717778e-01, 3.18876921e-01, 2.87598168e-01,
       2.13417145e-01, 1.37798256e-01, 1.37760712e-01, 1.22964576e-01,
       8.96866612e-02, 8.46246537e-02, 8.35736259e-02, 7.04887013e-02,
       5.81015262e-02, 5.33582445e-02, 5.19568548e-02, 5.05906906e-02,
       3.24017665e-02, 2.66982606e-02, 6.81418825e-03, 4.91074185e-03])

### GradientShap

In [17]:
gs = GradientShap(briskmodel_loaded)
# We define a distribution of baselines and draw `n_samples` from that
# distribution in order to estimate the expectations of gradients across all baselines
# baseline_dist = torch.randn(10, 3) * 0.001


In [18]:
# select a set of background examples to take an expectation over
background = X_tilda.loc[np.random.choice(X_tilda.shape[0], X_train.shape[0], replace=False)]
baseline_dist = helper.df_to_tensor(background)
# baseline_dist = baseline_dist.movedim(0,-1)


In [19]:
baseline_dist.shape, df_tensor.shape

(torch.Size([143, 40]), torch.Size([143, 40]))

In [20]:
gs_attr, delta = gs.attribute(df_tensor, stdevs=0.09, n_samples=1, baselines=baseline_dist, 
                                   # target=0, 
                                   return_convergence_delta=True
                                   )
# gs_attr = gs_attr.movedim(0,-1)

In [21]:
delta
# deltas_per_example = torch.mean(delta.reshape(df_tensor.shape[0], -1), dim=1)
# deltas_per_example

tensor([-1.8859e+06, -3.5696e+04,  5.8409e+06,  1.7903e+06, -2.7216e+05,
        -2.1504e+04,  5.1200e+02,  2.0890e+05, -4.2072e+05, -2.1278e+07,
         1.8135e+06,  2.2271e+06,  9.6870e+05, -8.6835e+05, -1.6778e+05,
        -3.3984e+04, -8.3578e+05,  2.1043e+05, -5.3111e+05,  1.3649e+07,
         5.8189e+05,  4.0525e+07,  2.8544e+06, -5.4630e+05, -1.3003e+05,
         8.7529e+06, -8.8504e+06, -1.4122e+06, -3.2000e+02, -3.9680e+03,
        -1.7096e+06,  6.5927e+07, -1.2736e+06,  2.8138e+07,  2.7600e+04,
        -2.8469e+05, -8.9664e+04,  1.9587e+06, -4.7494e+04, -8.0480e+05,
         2.4934e+05, -4.9792e+04,  4.0333e+05, -1.2481e+06,  7.4294e+05,
        -6.1568e+04,  2.4138e+06, -1.0650e+06,  3.7760e+03, -1.0778e+05,
         4.9792e+04,  7.5149e+05,  1.3292e+06,  2.6368e+04,  3.6915e+05,
         1.5337e+06, -4.9498e+05, -1.8803e+05, -1.8829e+05,  2.7443e+05,
         3.0720e+04, -5.6678e+05, -9.6928e+04, -1.9264e+04, -1.1151e+06,
         8.5807e+08, -7.1007e+07, -4.1882e+05,  2.5

In [22]:
gs_attr_df_shapley = pd.DataFrame(gs_attr.numpy(), columns=X_train.columns)
gs_attr_df_shapley.abs().mean().sort_values(ascending=False)
gs_attr_df_shapley_list = gs_attr_df_shapley.values

In [63]:
gs_attr_df_shapley = gs_attr_df_shapley.abs().mean().sort_values(ascending=False)

NoiseTunnel

In [24]:
nt: NoiseTunnel = NoiseTunnel(ig)

In [25]:
ig_attr_smoothgrad, delta = nt.attribute(df_tensor, nt_type='smoothgrad', stdevs=0.02, nt_samples=4,
      baselines=baseline_dist, 
      # target=0,
      return_convergence_delta=True)
# ig_attr_smoothgrad = ig_attr_smoothgrad.movedim(0,-1)

In [26]:
# delta
deltas_per_example = torch.mean(delta.reshape(df_tensor.shape[0], -1), dim=1)
deltas_per_example

tensor([ 7.6120e+04,  3.8367e+03,  1.0843e+04, -3.0455e+05,  1.0987e+04,
        -4.9414e+05,  4.4588e+04,  1.3596e+06, -2.8336e+04,  3.1601e+04,
        -3.0721e+04, -6.7004e+03, -1.7729e+04, -7.9002e+05,  1.0588e+04,
        -2.3662e+05, -4.8940e+04, -2.3230e+05, -1.9712e+03,  2.9382e+00,
         4.2722e+05, -2.2958e+03,  1.0794e+05, -5.7379e+03,  1.4122e+03,
        -1.4184e+04,  7.9511e+04, -1.7930e+05, -3.8917e+03, -1.1804e+05,
         2.2690e+02, -3.3703e+04,  1.8534e+05,  2.2974e+04,  2.5776e+03,
        -7.6995e+03, -4.8245e+02,  1.0646e+04, -1.2413e+03, -5.7671e+04,
        -6.2506e+04,  1.9601e+02,  8.3148e+04,  5.7073e+01,  6.1400e+03,
        -1.4662e+04,  4.7624e+01,  5.9224e+05,  1.5037e+04,  9.0320e+04,
        -4.3792e+02,  9.7345e+04, -3.8863e+05,  2.6600e+04,  4.8951e+04,
        -4.7666e+03, -8.3127e+03, -2.7977e+05,  6.1705e+04,  4.0584e+04,
         7.6506e+05,  7.8019e+04,  4.3575e+02,  2.5026e+04,  1.9493e+03,
         7.8734e+06, -6.9466e+05,  4.5567e+02,  1.6

In [27]:
ig_attr_smoothgrad_df_shapley = pd.DataFrame(ig_attr_smoothgrad.numpy(), columns=X_train.columns)
ig_attr_smoothgrad_score = ig_attr_smoothgrad_df_shapley.abs().mean().sort_values(ascending=False)
ig_attr_smoothgrad_score_list = ig_attr_smoothgrad_score.values

In [28]:
ig_attr_smoothgrad_score

Population                                    1.879347e+09
population_cov                                1.499498e+09
new_cases_per_million                         2.187344e+07
Area_km2                                      1.492912e+07
Density_km2                                   8.014394e+04
air_polution                                  4.520254e+02
life_expectancy_cov                           1.267414e+02
Cardiovascular diseases                       1.851219e+01
Total                                         1.322603e+01
Neoplasms                                     7.378679e+00
Diabetes mellitus                             4.205554e+00
Lower respiratory infections                  3.042008e+00
Alzheimer's disease and other dementias       2.935337e+00
HIV/AIDS                                      2.461959e+00
Cirrhosis and other chronic liver diseases    2.176194e+00
Chronic kidney disease                        1.946259e+00
Interpersonal violence                        1.286603e+

Shapley

In [29]:
from dask_ml.preprocessing import StandardScaler
X_train_scalar = StandardScaler().fit_transform(X_train)     
X_test_scalar = StandardScaler().fit_transform(X_test)     
X_test_tensor_scalar = helper.df_to_tensor(X_test_scalar)

In [30]:
background = X_tilda.sample(n=40)
background = helper.df_to_tensor(background)
X_tilda_tensor = helper.df_to_tensor(X_tilda)
# t = t.movedim(0,-1)

In [31]:
explainer_shap = shap.DeepExplainer(model=briskmodel_loaded, data=background)

In [32]:
explainer_shap.expected_value

array([1.1396457e+09], dtype=float32)

In [33]:
shap_values = explainer_shap.explainer.shap_values(X=X_tilda_tensor, ranked_outputs=True, check_additivity=False)

In [34]:
shap_values.shape

(191, 40)

In [43]:
df_shapley_sores = pd.DataFrame(shap_values, columns=X_train.columns)
df_shapley_sores = df_shapley_sores.abs().mean().sort_values(ascending=False)
df_shapley_sores_list = df_shapley_sores.values

In [53]:
df_shapley_sores

Population                                    1.156685e+09
population_cov                                9.209495e+08
new_cases_per_million                         1.724673e+07
Area_km2                                      1.268083e+07
Density_km2                                   4.366932e+04
air_polution                                  3.246251e+02
life_expectancy_cov                           8.492591e+01
Cardiovascular diseases                       1.314431e+01
Total                                         8.908080e+00
Neoplasms                                     5.429957e+00
Diabetes mellitus                             2.531922e+00
Alzheimer's disease and other dementias       1.937907e+00
Lower respiratory infections                  1.928483e+00
HIV/AIDS                                      1.739215e+00
Cirrhosis and other chronic liver diseases    1.284005e+00
Chronic kidney disease                        1.261377e+00
Chronic respiratory diseases                  7.600454e-

Consolidate

In [62]:
gs_attr_df_shapley

,new_cases_per_million,population_cov,life_expectancy_cov,Population,Area_km2,Density_km2,Year_x,Meningitis,Neoplasms,"Fire, heat, and hot substances",...,Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease,Total,air_polution
0,16550438.0,-4.484640e+06,5.906300,-8.196456e+06,1.060079e+05,-41302.171875,3.255215,-0.179583,4.377513,-0.794008,...,22.041592,-2.433782,0.388203,-2.661170,-2.764581,-0.255335,1.929859,-0.493741,20.741268,509.531738
1,57008552.0,5.238617e+07,69.390129,4.023664e+07,-1.256630e+05,-12373.023438,2.668525,-3.569902,7.403769,-0.356330,...,28.342234,-3.321461,2.154446,6.969543,-1.095488,-2.486426,13.535871,3.990681,19.663357,22.533657
2,12294561.0,-3.598962e+10,-184.299454,-3.412089e+10,8.454836e+07,14391.077148,0.039496,-2.935073,-9.350083,0.327694,...,-15.545536,-10.390826,-4.187019,-14.180088,0.269781,-0.371997,4.237294,-1.207724,3.436200,-424.617706
3,7704053.5,5.155417e+08,145.906647,8.143690e+08,-2.014416e+08,26853.361328,-0.391325,1.215265,0.581771,-0.161989,...,11.876887,-7.265506,-4.875596,9.530360,-0.346214,-1.219251,16.885433,3.173777,-0.470746,-336.443634
4,-29025762.0,-2.011686e+08,-146.692993,-2.402939e+08,5.936608e+06,-36460.128906,0.003006,-0.441868,-10.148687,-0.752397,...,3.979063,-1.212159,7.783840,-1.416136,-0.105616,0.106488,-20.093790,2.034074,-4.262027,496.550293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,10718389.0,-2.325803e+07,23.639860,-2.742769e+07,-3.152113e+05,40630.468750,-0.027799,1.594040,-0.544716,-1.912644,...,-21.488388,15.110810,-0.764059,7.104227,-0.177468,0.231822,6.146091,3.588944,-12.119619,-44.097107
139,58867828.0,6.337262e+07,-117.238548,7.612578e+07,-1.017279e+06,11030.140625,0.984067,0.635954,7.742444,-0.816382,...,43.944496,17.853474,12.100704,-0.045651,-0.041799,-0.188337,19.831568,-1.787470,30.555412,119.075821
140,-18452058.0,4.181700e+06,4.647046,8.075586e+06,-5.756658e+05,4627.017090,-0.155336,0.934519,-0.858969,0.025824,...,16.024883,1.232855,-2.772782,8.277256,0.073663,-0.213558,9.953946,-0.093592,6.703205,158.327667
141,11252040.0,-2.031452e+08,-15.621056,-2.878174e+08,1.997244e+06,-1469.446533,0.008479,4.012170,22.090120,-0.166621,...,36.801270,-5.487327,10.059072,-9.798722,0.040878,-0.296612,14.755261,2.403800,27.763762,52.703892


In [65]:
df_consol = pd.DataFrame([ig_attr_df_shapley, gs_attr_df_shapley, ig_attr_smoothgrad_score, df_shapley_sores])

In [67]:
df_consol = df_consol.T

In [68]:
df_consol

,0,1,2,3
Population,1.022733e+09,1.993401e+09,1.879347e+09,1.156685e+09
population_cov,8.476716e+08,1.780833e+09,1.499498e+09,9.209495e+08
new_cases_per_million,2.239716e+07,2.265344e+07,2.187344e+07,1.724673e+07
Area_km2,1.033267e+07,1.679242e+07,1.492912e+07,1.268083e+07
Density_km2,7.186737e+04,5.093695e+04,8.014394e+04,4.366932e+04
Year_x,2.196134e+04,6.202123e-01,6.645190e-02,0.000000e+00
life_expectancy_cov,1.162633e+03,1.292580e+02,1.267414e+02,8.492591e+01
air_polution,5.986486e+02,4.614802e+02,4.520254e+02,3.246251e+02
Total,3.593387e+01,1.304012e+01,1.322603e+01,8.908080e+00
Cardiovascular diseases,2.809641e+01,2.053753e+01,1.851219e+01,1.314431e+01


In [ ]:
df_consol.columns = ['IG', 'GS', 'IG_SMOOTHED', 'SHAPLEY']

DeepLift

In [115]:
dl = DeepLift(briskmodel_loaded)
attributions, delta = dl.attribute(df_tensor, baseline_dist, 
                                        # target=0,
                                        return_convergence_delta=True)


RuntimeError: Cannot use both regular backward hooks and full backward hooks on a single Module. Please use only one of them.

DeepLiftShap

In [114]:
dl = DeepLiftShap(briskmodel_loaded)
dl_attr, delta = dl.attribute(df_tensor, baseline_dist, 
                                                # target=0,
                                                return_convergence_delta=True)


NameError: name 'DeepLiftShap' is not defined

In [85]:
# test = torch.tensor(X_test.values.astype(np.float32))
X_test_tensor = torch.tensor(X_train.values).float()

In [79]:
X_test_tensor.shape

torch.Size([1, 143, 40])